Индивидальный проект

Название проекта: Информация о деталях заказа

Вариант задания  № 15

Описание:

Создать базовый класс OrderLine в C#, который будет представлять информацию о
строке заказа, содержащей детали одного товара в заказе. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу OrderLine:

• Атрибуты: ID товара (ProductId), Название товара (ProductName), Цена
товара (Price).

• Методы:

• CalculateTotal(): метод для расчета общей стоимости строки заказа.

• UpdatePrice(decimal newPrice): метод для обновления цены товара в
строке заказа.

• GetProductDetails(): метод для получения деталей товара.

Требования к производным классам:
1. СтандартнаяСтрока (StandardLine): Должна содержать дополнительные
атрибуты, такие как Количество единиц (Units). Метод CalculateTotal() должен
быть переопределен для учета количества единиц при расчете общей
стоимости.
2. СпециальнаяСтрока (SpecialLine): Должна содержать дополнительные
атрибуты, такие как Скидка (Discount). Метод UpdatePrice() должен быть
переопределен для применения скидки к цене товара.
3. БесплатнаяСтрока (FreeLine) (если требуется третий класс): Должна
содержать дополнительные атрибуты, такие как Предварительный платеж
(Prepayment). Метод CalculateTotal() должен быть переопределен для учета
предварительного плата при расчете общей стоимости.

Дополнительное задание:

Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями.

Реализация:

In [2]:

interface IReservable
{
    void Reserve();
}

interface IDiscountable
{
    void ApplyDiscount(decimal discountAmount);
}

interface IPrepayable
{
    void ProcessPrepayment(decimal prepaymentAmount);
}

class OrderLine
{
    public string Category { get; set; }
    public string Description { get; set; }
    public string Manufacturer { get; set; }
    public int WarrantyPeriod { get; set; }
    public string DiscountCode { get; set; }
    protected int ProductId { get; set; }
    protected string ProductName { get; set; }
    protected decimal Price { get; set; }
    protected int StockQuantity { get; set; }
    protected bool IsAvailable { get; set; }
    protected int Rating { get; set; }

    public virtual bool IsInStock() => StockQuantity > 0;

    public void ApplyDiscountCode(string code)
    {
        DiscountCode = code;
        Console.WriteLine($"Код скидки {code} применён.");
    }

    public string GetWarrantyStatus() =>
        WarrantyPeriod > 0 ? "Гарантия действительна" : "Нет гарантии";

    public virtual decimal CalculateTotal() => Price;

    public virtual string GetProductDetails() =>
        $"ProductId: {ProductId}, ProductName: {ProductName}, Цена: {Price}, Гарантия: {GetWarrantyStatus()}";

    public OrderLine(int productId, string productName, decimal price, string category, string description, string manufacturer, int warrantyPeriod)
    {
        ProductId = productId;
        ProductName = productName;
        Price = price;
        Category = category;
        Description = description;
        Manufacturer = manufacturer;
        WarrantyPeriod = warrantyPeriod;
    }
}

class OrderLineWithDiscount : OrderLine, IDiscountable
{
    protected decimal Discount { get; set; }
    protected DateTime ValidUntil { get; set; }

    public OrderLineWithDiscount(int productId, string productName, decimal price, string category, string description, string manufacturer, int warrantyPeriod, decimal discount, DateTime validUntil)
        : base(productId, productName, price, category, description, manufacturer, warrantyPeriod)
    {
        Discount = discount;
        ValidUntil = validUntil;
    }

    public void ApplyDiscount(decimal discountAmount)
    {
        Price -= discountAmount;
        Console.WriteLine($"Скидка для {discountAmount} применена. Новая стоимость: {Price}");
    }

    public void CheckDiscountValidity()
    {
        Console.WriteLine(DateTime.Now < ValidUntil ? "Скидка действительна." : "Срок действия скидки истек.");
    }

    public override string GetProductDetails() =>
        base.GetProductDetails() + $", Скидка: {Discount}, Действует до: {ValidUntil}";
}

class StandertLine : OrderLineWithDiscount
{
    protected int Units { get; set; }
    protected string BatchNumber { get; set; }

    public StandertLine(int productId, string productName, decimal price, string category, string description, string manufacturer, int warrantyPeriod, decimal discount, DateTime validUntil, int units, string batchNumber)
        : base(productId, productName, price, category, description, manufacturer, warrantyPeriod, discount, validUntil)
    {
        Units = units;
        BatchNumber = batchNumber;
    }

    public override decimal CalculateTotal() => Price * Units;

    public void CheckBatchAvailability() => Console.WriteLine($"Партия {BatchNumber} наличие подтверждено.");

    public override string GetProductDetails() =>
        base.GetProductDetails() + $", Единицы: {Units}, Партия: {BatchNumber}";
}

class SpecialLine : OrderLineWithDiscount, IReservable
{
    protected string ReservationId { get; set; }

    public SpecialLine(int productId, string productName, decimal price, string category, string description, string manufacturer, int warrantyPeriod, decimal discount, DateTime validUntil)
        : base(productId, productName, price, category, description, manufacturer, warrantyPeriod, discount, validUntil)
    {
        GenerateReservationId();
    }

    private void GenerateReservationId() => ReservationId = Guid.NewGuid().ToString();
    void IReservable.Reserve() => Console.WriteLine($"Забронировано {ProductName} with ID {ReservationId}");

    public override string GetProductDetails() =>
        base.GetProductDetails() + $", ID бронирования: {ReservationId}";
   
    void IReservable()
    {
        Console.WriteLine($"Забронировано {ProductName}");
    }
    
}

class FreeLine : OrderLineWithDiscount, IReservable, IPrepayable
{
    protected decimal Prepayment { get; set; }
    protected decimal CancellationFee { get; set; }

    public FreeLine(int productId, string productName, decimal price, string category, string description, string manufacturer, int warrantyPeriod, decimal discount, DateTime validUntil, decimal prepayment, decimal cancellationFee)
        : base(productId, productName, price, category, description, manufacturer, warrantyPeriod, discount, validUntil)
    {
        Prepayment = prepayment;
        CancellationFee = cancellationFee;
    }

    public override decimal CalculateTotal() => Price - Prepayment;

    void IPrepayable.ProcessPrepayment(decimal prepaymentAmount)
    {
        Prepayment = prepaymentAmount;
        Console.WriteLine($"Обработана предоплата в размере {prepaymentAmount} для {ProductName}");
    }

    void IReservable.Reserve() => Console.WriteLine($"Забронировано {ProductName} с предоплатой в размере {Prepayment}");

    public override string GetProductDetails() =>
        base.GetProductDetails() + $", Предоплата: {Prepayment}, Плата за отмену: {CancellationFee}";

        void IReservable()
    {
        Console.WriteLine($"Забронировано {ProductName}");
    }

}

StandertLine standertLine = new StandertLine(1, "ProductA", 10, "Электроника", "Смартфон", "Samsung", 24, 2, DateTime.Now.AddMonths(1), 5, "Партия123");

SpecialLine specialLine = new SpecialLine(2, "ProductB", 15, "Бытовая техника", "Блендер", "Philips", 12, 3, DateTime.Now.AddMonths(3));

FreeLine freeLine = new FreeLine(3, "ProductC", 20, "Игрушки", "Игрушечная машина", "Lego", 6, 10, DateTime.Now.AddMonths(6), 5, 2);

standertLine.ApplyDiscount(1); 
standertLine.CheckBatchAvailability();  

((IPrepayable)freeLine).ProcessPrepayment(5);


Скидка для 1 применена. Новая стоимость: 9
Партия Партия123 наличие подтверждено.
Обработана предоплата в размере 5 для ProductC
